In [1]:
import os
import struct
import portalocker
import pickle

In [5]:
# Connect to DB

class it_DBDB(object):

    def __init__(self, f):
        self._storage = Storage(f)
        self._tree = BinaryTree(self._storage)

    def _assert_not_closed(self):
        if self._storage.closed:
            raise ValueError('Database closed.')

    def close(self):
        self._storage.close()

    def commit(self):
        self._assert_not_closed()
        self._tree.commit()

    def get(self, key):
        self._assert_not_closed()
        return self._tree.get(key)

    def set(self, key, value):
        self._assert_not_closed()
        return self._tree.set(key, value)

    def delete(self, key):
        self._assert_not_closed()
        return self._tree.delete(key)

def it_connect(dbname):
    try:
        f = open(dbname, 'r+b')
    except IOError:
        fd = os.open(dbname, os.O_RDWR | os.O_CREAT)
        f = os.fdopen(fd, 'r+b')
    return it_DBDB(f)

In [6]:
def connect(dbname):
    try:
        f = open(dbname, 'r+b')
    except IOError:
        fd = os.open(dbname, os.O_RDWR | os.O_CREAT)
        f = os.fdopen(fd, 'r+b')
    return DBDB(f)


class DBDB(object):

    def __init__(self, f):
        self._storage = Storage(f)
        self._tree = RedBlackTree(self._storage)

    def _assert_not_closed(self):
        if self._storage.closed:
            raise ValueError('Database closed.')

    def close(self):
        self._storage.close()

    def commit(self):
        self._assert_not_closed()
        self._tree.commit()

    def get(self, key):
        self._assert_not_closed()
        return self._tree.get(key)

    def set(self, key, value):
        self._assert_not_closed()
        return self._tree.set(key, value)

    def getRootKey(self):
        return self._tree.rootkey()

    def delete(self, key):
        self._assert_not_closed()
        return self._tree.delete(key)

In [7]:
# Tree Structure

class Storage(object):
    SUPERBLOCK_SIZE = 4096
    INTEGER_FORMAT = "!Q"
    INTEGER_LENGTH = 8

    def __init__(self, f):
        self._f = f
        self.locked = False
        #we ensure that we start in a sector boundary
        self._ensure_superblock()

    def _ensure_superblock(self):
        "guarantee that the next write will start on a sector boundary"
        self.lock()
        self._seek_end()
        end_address = self._f.tell()
        if end_address < self.SUPERBLOCK_SIZE:
            self._f.write(b'\x00' * (self.SUPERBLOCK_SIZE - end_address))
        self.unlock()

    def lock(self):
        "if not locked, lock the file for writing"
        if not self.locked:
            portalocker.lock(self._f, portalocker.LOCK_EX)
            self.locked = True
            return True
        else:
            return False

    def unlock(self):
        if self.locked:
            self._f.flush()
            portalocker.unlock(self._f)
            self.locked = False

    def _seek_end(self):
        self._f.seek(0, os.SEEK_END)

    def _seek_superblock(self):
        "go to beginning of file which is on sec boundary"
        self._f.seek(0)

    def _bytes_to_integer(self, integer_bytes):
        return struct.unpack(self.INTEGER_FORMAT, integer_bytes)[0]

    def _integer_to_bytes(self, integer):
        return struct.pack(self.INTEGER_FORMAT, integer)

    def _read_integer(self):
        return self._bytes_to_integer(self._f.read(self.INTEGER_LENGTH))

    def _write_integer(self, integer):
        self.lock()
        self._f.write(self._integer_to_bytes(integer))

    def write(self, data):
        "write data to disk, returning the adress at which you wrote it"
        #first lock, get to end, get address to return, write size
        #write data, unlock <==WRONG, dont want to unlock here
        #your code here
        self.lock()
        self._seek_end()
        object_address = self._f.tell()
        self._write_integer(len(data))
        self._f.write(data)
        return object_address

    def read(self, address):
        self._f.seek(address)
        length = self._read_integer()
        data = self._f.read(length)
        return data

    def commit_root_address(self, root_address):
        self.lock()
        self._f.flush()
        #make sure you write root address at position 0
        self._seek_superblock()
        #write is atomic because we store the address on a sector boundary.
        self._write_integer(root_address)
        self._f.flush()
        self.unlock()

    def get_root_address(self):
        #read the first integer in the file
        #your code here
        self._seek_superblock()
        root_address = self._read_integer()
        return root_address

    def close(self):
        self.unlock()
        self._f.close()

    @property
    def closed(self):
        return self._f.closed
        

In [8]:
# Tree logic

class ValueRef(object):
    " a reference to a string value on disk"
    def __init__(self, referent=None, address=0):
        self._referent = referent #value to store
        self._address = address #address to store at
        
    @property
    def address(self):
        return self._address
    
    def prepare_to_store(self, storage):
        pass

    @staticmethod
    def referent_to_bytes(referent):
        return referent.encode('utf-8')

    @staticmethod
    def bytes_to_referent(bytes):
        return bytes.decode('utf-8')

    
    def get(self, storage):
        "read bytes for value from disk"
        if self._referent is None and self._address:
            self._referent = self.bytes_to_referent(storage.read(self._address))
        return self._referent

    def store(self, storage):
        "store bytes for value to disk"
        #called by BinaryNode.store_refs
        if self._referent is not None and not self._address:
            self.prepare_to_store(storage)
            self._address = storage.write(self.referent_to_bytes(self._referent))

class BinaryNodeRef(ValueRef):
    "reference to a btree node on disk"

    #calls the BinaryNode's store_refs
    def prepare_to_store(self, storage):
        "have a node store its refs"
        if self._referent:
            self._referent.store_refs(storage)

    @staticmethod
    def referent_to_bytes(referent):
        "use pickle to convert node to bytes"
        return pickle.dumps({
            'left': referent.left_ref.address,
            'key': referent.key,
            'value': referent.value_ref.address,
            'right': referent.right_ref.address,
        })

    @staticmethod
    def bytes_to_referent(string):
        "unpickle bytes to get a node object"
        d = pickle.loads(string)
        return BinaryNode(
            BinaryNodeRef(address=d['left']),
            d['key'],
            ValueRef(address=d['value']),
            BinaryNodeRef(address=d['right']),
        )
              
class BinaryNode(object):
    @classmethod
    def from_node(cls, node, **kwargs):
        "clone a node with some changes from another one"
        return cls(
            left_ref=kwargs.get('left_ref', node.left_ref),
            key=kwargs.get('key', node.key),
            value_ref=kwargs.get('value_ref', node.value_ref),
            right_ref=kwargs.get('right_ref', node.right_ref),
        )

    def __init__(self, left_ref, key, value_ref, right_ref):
        self.left_ref = left_ref
        self.key = key
        self.value_ref = value_ref
        self.right_ref = right_ref

    def store_refs(self, storage):
        "method for a node to store all of its stuff"
        self.value_ref.store(storage)
        #calls BinaryNodeRef.store. which calls
        #BinaryNodeRef.prepate_to_store
        #which calls this again and recursively stores
        #the whole tree
        self.left_ref.store(storage)
        self.right_ref.store(storage)
        

In [9]:
# BST Implementation

class BinaryTree(object):
    "Immutable Binary Tree class. Constructs new tree on changes"
    def __init__(self, storage):
        self._storage = storage
        self._refresh_tree_ref()

    def commit(self):
        "changes are final only when committed"
        #triggers BinaryNodeRef.store
        self._tree_ref.store(self._storage)
        #make sure address of new tree is stored
        self._storage.commit_root_address(self._tree_ref.address)

    def _refresh_tree_ref(self):
        "get reference to new tree if it has changed"
        self._tree_ref = BinaryNodeRef(
            address=self._storage.get_root_address())

    def get(self, key):
        "get value for a key"
        #if tree is not locked by another writer
        #refresh the references and get new tree if needed
        if not self._storage.locked:
            self._refresh_tree_ref()
        #get the top level node
        node = self._follow(self._tree_ref)
        #traverse until you find appropriate node
        while node is not None:
            if key < node.key:
                node = self._follow(node.left_ref)
            elif key > node.key:
                node = self._follow(node.right_ref)
            else:
                return self._follow(node.value_ref)
        raise KeyError

    def set(self, key, value):
        "set a new value in the tree. will cause a new tree"
        #try to lock the tree. If we succeed make sure
        #we dont lose updates from any other process
        if self._storage.lock():
            self._refresh_tree_ref()
        #get current top-level node and make a value-ref
        node = self._follow(self._tree_ref)
        value_ref = ValueRef(value)
        #insert and get new tree ref
        self._tree_ref = self._insert(node, key, value_ref)


    def _insert(self, node, key, value_ref):
        "insert a new node creating a new path from root"
        #create a tree if there was none so far
        if node is None:
            new_node = BinaryNode(
                BinaryNodeRef(), key, value_ref, BinaryNodeRef())
        elif key < node.key:
            new_node = BinaryNode.from_node(
                node,
                left_ref=self._insert(
                    self._follow(node.left_ref), key, value_ref))
        elif key > node.key:
            new_node = BinaryNode.from_node(
                node,
                right_ref=self._insert(
                    self._follow(node.right_ref), key, value_ref))
        else: #create a new node to represent this data
            new_node = BinaryNode.from_node(node, value_ref=value_ref)
        return BinaryNodeRef(referent=new_node)

    def delete(self, key):
        "delete node with key, creating new tree and path"
        if self._storage.lock():
            self._refresh_tree_ref()
        node = self._follow(self._tree_ref)
        self._tree_ref = self._delete(node, key)

    def _delete(self, node, key):
        "underlying delete implementation"
        if node is None:
            raise KeyError
        elif key < node.key:
            new_node = BinaryNode.from_node(
                node,
                left_ref=self._delete(
                    self._follow(node.left_ref), key))
        elif key > node.key:
            new_node = BinaryNode.from_node(
                node,
                right_ref=self._delete(
                    self._follow(node.right_ref), key))
        else:
            left = self._follow(node.left_ref)
            right = self._follow(node.right_ref)
            if left and right:
                replacement = self._find_max(left)
                left_ref = self._delete(
                    self._follow(node.left_ref), replacement.key)
                new_node = BinaryNode(
                    left_ref,
                    replacement.key,
                    replacement.value_ref,
                    node.right_ref,
                )
            elif left:
                return node.left_ref
            else:
                return node.right_ref
        return BinaryNodeRef(referent=new_node)

    def _follow(self, ref):
        "get a node from a reference"
        #calls BinaryNodeRef.get
        return ref.get(self._storage)

    def _find_max(self, node):
        while True:
            next_node = self._follow(node.right_ref)
            if next_node is None:
                return node
            node = next_node

In [10]:
class Color(object):
    RED = 0
    BLACK = 1

class RedBlackNodeRef(ValueRef):
    " a reference to a string value on disk"
    def __init__(self, referent=None, address=0):
        self._referent = referent #value to store
        self._address = address #address to store at

    "reference to a btree node on disk"

    #calls the BinaryNode's store_refs
    def prepare_to_store(self, storage):
        "have a node store its refs"
        if self._referent:
            self._referent.store_refs(storage)

    @staticmethod
    def referent_to_bytes(referent):
        "use pickle to convert node to bytes"
        return pickle.dumps({
            'left': referent.left_ref.address,
            'key': referent.key,
            'value': referent.value_ref.address,
            'right': referent.right_ref.address,
            'color': referent.color
        })

    @staticmethod
    def bytes_to_referent(string):
        "unpickle bytes to get a node object"
        d = pickle.loads(string)
        return RedBlackNode(
            RedBlackNodeRef(address=d['left']),
            d['key'],
            ValueRef(address=d['value']),
            RedBlackNodeRef(address=d['right']),
            d['color']
        )


class RedBlackNode(BinaryNode):

    @classmethod
    def from_node(cls, node, **kwargs):
        "clone a node with some changes from another one"
        return cls(
            left_ref=kwargs.get('left_ref', node.left_ref),
            key=kwargs.get('key', node.key),
            value_ref=kwargs.get('value_ref', node.value_ref),
            right_ref=kwargs.get('right_ref', node.right_ref),
            color=kwargs.get('color', node.color),
        )

    def __init__(self, left_ref, key, value_ref, right_ref, color=Color.RED):
        self.left_ref = left_ref
        self.key = key
        self.value_ref = value_ref
        self.right_ref = right_ref
        self.color = color

    def is_black(self):
        return self.color == Color.BLACK

    def is_red(self):
        return self.color == Color.RED

    def is_empty(self):
        return False


class RedBlackTree(BinaryTree):

    def _refresh_tree_ref(self):
        "get reference to new tree if it has changed"
        self._tree_ref = RedBlackNodeRef(
            address=self._storage.get_root_address())

    def is_empty(self):
        return False

    def _blacken(self, node):
        #node = self._follow(ref)
        if node is None:
            return node
        newnode = RedBlackNode.from_node(node, color=Color.BLACK)
        return RedBlackNodeRef(newnode)

    def rootkey(self):
        return self._follow(self._tree_ref).key

    def rotate_left(self, node):
        right = self._follow(node.right_ref)
        if self._follow(right.left_ref) is not None:
            newleft_right = RedBlackNode.from_node(self._follow(right.left_ref),
                                                   left_ref = RedBlackNodeRef(),
                                                   right_ref = RedBlackNodeRef(),
                                                   color = node.color)
            newleft_right_ref = RedBlackNodeRef(referent = newleft_right)
        else:
            newleft_right_ref = RedBlackNodeRef()
        newleft = RedBlackNode.from_node(node, 
                                         right_ref = newleft_right_ref)
        newnode = RedBlackNode.from_node(right,
                                         left_ref = RedBlackNodeRef(referent=newleft))
        return newnode


    def rotate_right(self, node):
        left = self._follow(node.left_ref)
        if self._follow(left.right_ref) is not None:
            newright_left = RedBlackNode.from_node(self._follow(left.right_ref),
                                                   left_ref = RedBlackNodeRef(),
                                                   right_ref = RedBlackNodeRef(),
                                                   color = node.color)
            newright_left_ref = RedBlackNodeRef(referent = newleft_right)
        else:
            newright_left_ref = RedBlackNodeRef()
                                        
        newright = RedBlackNode.from_node(node, 
                                          left_ref = newright_left_ref)
        newnode = RedBlackNode.from_node(left,
                                         right_ref = RedBlackNodeRef(referent=newright))

        return newnode
                                      
    def recolor(self, node):
        left = self._blacken(self._follow(node.left_ref))
        right = self._blacken(self._follow(node.right_ref))

        return RedBlackNode.from_node(node, 
                                      left_ref = left,
                                      right_ref = right,
                                      color=Color.RED)

    def _isred(self, node):
        if node is None:
            return False
        else:
            return node.color == Color.RED

    def balance(self, node):

        if self._isred(node) | (node is None):
            return node

        left = self._follow(node.left_ref)
        right = self._follow(node.right_ref)

        if self._isred(left):
            if self._isred(right):
                return self.recolor(node)
            if self._isred(self._follow(left.left_ref)):
                return self.recolor(self.rotate_right(node))
            if self._isred(self._follow(left.right_ref)):
                newleft = self.rotate_left(left)
                newnode = RedBlackNode.from_node(node, 
                                                 left_ref = RedBlackNodeRef(referent = newleft))
                return self.recolor(self.rotate_right(newnode))
        if self._isred(right):
            if self._isred(self._follow(right.right_ref)):
                return self.recolor(self.rotate_left(node))
            if self._isred(self._follow(right.left_ref)):
                newright = self.rotate_right(right)
                newnode = RedBlackNode.from_node(node, 
                                                 right_ref = RedBlackNodeRef(referent = newright))
                return self.recolor(self.rotate_left(newnode))

        return node

    def set(self, key, value):
        "set a new value in the tree. will cause a new tree"
        #try to lock the tree. If we succeed make sure
        #we dont lose updates from any other process
        if self._storage.lock():
            self._refresh_tree_ref()
        #get current top-level node and make a value-ref
        node = self._follow(self._tree_ref)
        value_ref = ValueRef(value)
        #insert and get new tree ref
        self._tree_ref = self._insert(node, key, value_ref)
        self._tree_ref = self._blacken(self._follow(self._tree_ref))


    def _insert(self, node, key, value_ref):
        "insert a new node creating a new path from root"
        #create a tree if there was none so far
        if node is None:
            #print ('a')
            new_node = RedBlackNode(
               RedBlackNodeRef(), key, value_ref, RedBlackNodeRef())
        elif key < node.key:
            newleft_ref = self._insert(self._follow(node.left_ref), key, value_ref)
            newleft = self.balance(self._follow(newleft_ref))
            new_node = self.balance(RedBlackNode.from_node(
                    node,
                    left_ref=RedBlackNodeRef(referent=newleft)))
        elif key > node.key:
            newright_ref = self._insert(self._follow(node.right_ref), key, value_ref)
            newright = self.balance(self._follow(newright_ref))
            new_node = self.balance(RedBlackNode.from_node(
                    node,
                    right_ref=RedBlackNodeRef(referent=newright)))
        else: #create a new node to represent this data
            new_node = RedBlackNode.from_node(node, value_ref=value_ref)
        #new_node = self._blacken(new_node)
        return RedBlackNodeRef(referent=new_node)


In [22]:
import subprocess
from IPython.display import Image
import os.path
def plotbtree(btree, fname, redo=False, augmented=False):
    if os.path.exists(fname+".png") and not redo:
        return Image(fname+".png")
    start="""
    digraph G {
    nodesep=0.3;
    ranksep=0.2;
    margin=0.1;
    node [shape=circle];
    edge [arrowsize=0.8];
    """
    end = "}"
    assert btree.isRoot(), "Must start at root"
    lines=""
    labeldict={}
    for t in btree.preorder():
        if not t[0].uuid in labeldict:
            labeldict[t[0].uuid]=t[0]
        if not t[1].uuid in labeldict:
            labeldict[t[1].uuid]=t[1]
        lines = lines + "\"{}\" -> \"{}\"[side={}];\n".format(t[0].uuid,t[1].uuid,t[2])
    if augmented:
        end="\n".join(["\"{}\"[label=<{}<BR/><FONT POINT-SIZE=\"10\">{},{}</FONT>>]".format(k,v.data,v.size, v.count) for (k, v) in labeldict.items()])+end
    else:
        end="\n".join(["\"{}\"[label=<{}<BR/><FONT POINT-SIZE=\"10\">count={}</FONT>>]".format(k,v.data,v.count) for (k, v) in labeldict.items()])+end
       
    start = start + lines +end
    with open(fname+".dot","w") as fd:
        fd.write(start)
    subprocess.call(['./treeplot.sh', fname])
    return Image(fname+".png")

In [31]:
db = connect("/tmp/test1.dbdb")

In [32]:
db.set("rahul",25)
db.set("omar",66)
db.set("virgile",1)
db.set("heidi",3)

In [39]:
print(db._tree)